In [1]:
from datasets import load_dataset, load_metric, ClassLabel, Value
# load data into train and test set
dataset = load_dataset('csv', data_files={'train': 'processed_data.csv',
                                          'test': 'processed_data.csv'})

Using custom data configuration default-5fcccda6ed180e52


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\aylat\.cache\huggingface\datasets\csv\default-5fcccda6ed180e52\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
dataset["train"][0] # example instance from the loaded dataset

{'target': 'cool',
 'sentence': "It 's so cool to see how such old software can still run under Windows 10 .",
 'label': 'cool_1'}

In [3]:
# define the labels for classification
sense_labels = ["cool_1", "cool_2"]
class_labels = ClassLabel(num_classes=len(sense_labels), names=list(sense_labels))
class_labels

ClassLabel(num_classes=2, names=['cool_1', 'cool_2'], names_file=None, id=None)

In [4]:
def label_str2int(examples):
    # map the label string to integer
    return {"label": class_labels.str2int(examples["label"])}

# convert labels to integer ids
dataset = dataset.map(label_str2int, batched=False)

  0%|          | 0/2 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ex/s]

In [5]:
dataset["train"][0] # example instance after label conversion

{'target': 'cool',
 'sentence': "It 's so cool to see how such old software can still run under Windows 10 .",
 'label': 0}

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
# load pretrained transformer and corresponding tokenizer
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=class_labels.num_classes)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [7]:
def encode(examples):
    return tokenizer(examples['sentence'], truncation=True, padding="max_length", max_length=128)

# tokenize/encode input sentence
encoded_dataset = dataset.map(encode, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
new_features = encoded_dataset["train"].features.copy()
new_features["label"] = class_labels
encoded_dataset = encoded_dataset.cast(new_features)  # cast label from int to ClassLabel

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
import numpy as np
f1_metric = load_metric("f1")

# define evaluation metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1_metric.compute(predictions=predictions, references=labels, average="micro")

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [12]:
# define training details
args = TrainingArguments(
    "sense-disambiguation",  # model output path
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
#     fp16=True,
)

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics
)

In [14]:
# start training
# trainer.args._n_gpu =1
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, target.
***** Running training *****
  Num examples = 2
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5, training_loss=0.5698206424713135, metrics={'train_runtime': 6.3533, 'train_samples_per_second': 1.574, 'train_steps_per_second': 0.787, 'total_flos': 331168496640.0, 'train_loss': 0.5698206424713135, 'epoch': 5.0})